In [1]:
#2018-12-05 20:39:04.249461
%load_ext metapack.jupyter.magic

In [2]:
CACHE_DIR='/Users/eric/Library/Application Support/metapack/'
RESOURCE_NAME='rasp_tracts_sd'
RESOLVED_URL='file:///Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiegodata.org/sandiegodata.org-rasp/notebooks/RaspTracts.ipynb#df'
WORKING_DIR='/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiegodata.org/sandiegodata.org-rasp'
METATAB_DOC='metapack+file:///Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiegodata.org/sandiegodata.org-rasp/metadata.csv'
METATAB_WORKING_DIR='/Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiegodata.org/sandiegodata.org-rasp'
METATAB_PACKAGE='metapack+file:///Users/eric/proj/virt-proj/data-project/sdrdl-data-projects/sandiegodata.org/sandiegodata.org-rasp/'
name='rasp_tracts_sd'
description='Population estimates by race, age, sex and poverty status, for San Diego county'
url='notebooks/RaspTracts.ipynb#df'

In [3]:
METAPACK_BUILDING=True

In [4]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display 
from tqdm import tqdm_notebook

%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [5]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

In [6]:
race_iterations = pkg.reference('race_iterations').dataframe().fillna('') # All race's empty code becomes NaN

# List of PUMS in San Giego county, so we can restrict census data to just SD County
sandiego_puma_tract =  pkg.reference('sandiego_puma_tract').dataframe()
sd_pumas = list(sandiego_puma_tract.puma_geoid.unique())
sd_tracts = list(sandiego_puma_tract.tract_geoid.unique())
sd_puma_tracts = sd_pumas + sd_tracts

In [7]:
# Get the sum of the population of all PUMS
from rowgenerators import parse_app_url
t = parse_app_url('census://2016/5/CA/795/B01003').dataframe()
t = t[t.index.isin(sd_pumas)]
sd_puma_pop = t.B01003_001.sum()

# Get the controlled population for SD county. THe margin for this estimate is zero
# because it's tied to administrative data. 
from rowgenerators import parse_app_url
t = parse_app_url('census://2016/5/CA/50/B01003').dataframe()
sd_controlled_pop = t.loc['05000US06073'].B01003_001 

# Apparently, the PUMA population is also controlles. 
assert sd_puma_pop == sd_controlled_pop

# However, the sums from the poverty status by sex by age table is not
# controlled ( and would not expect it to be ) so it's a different number. 
#
# Interestingly, the controlled population number is not with in the 90% margins of 
# the sum of the table esimates. 
from rowgenerators import parse_app_url
t = parse_app_url('census://2016/5/CA/795/B17001').dataframe()
t = t[t.index.isin(sd_pumas)]
t.B17001_001.sum(), t.B17001_001.sum() - sd_controlled_pop, t.B17001_001.sum_m90()

(3172544, -80812, 9788.614048985688)

In [8]:
def make_col_map(t):
    from collections import namedtuple

    sex = None
    age = None
    pov = 'all'

    def parse_age(v):
        
        if v is None:
            return None
        
        elif v and 'year' in v:
            parts = v.split()
            
            if parts[0] == '75':
                return  pd.Interval(left=75, right=120, closed='both')
            elif parts[0] == '5':
                return  pd.Interval(left=5, right=5, closed='both')
            elif parts[0] == '15':
                return  pd.Interval(left=15, right=15, closed='both')
            elif parts[0] == 'Under':
                return  pd.Interval(left=0, right=4, closed='both')
            elif len(parts) == 4:
                return  pd.Interval(left=int(parts[0]), right=int(parts[2]), closed='both') 
            else:
                raise ValueError("Can't parse age string: {}".format(v))
        else:
            return  pd.Interval(left=0, right=120, closed='both') 

    col_map = {}

    Dimensions = namedtuple('Dimensions', ['sex', 'age_range', 'pov'])

    for e in t.table.columns:

        sd = e.short_description
        if '_m90' in e.unique_id:
            continue
        if sd:
            if 'Male' in sd:
                sex = 'male'
            elif 'Female' in sd:
                sex = 'female'
            elif 'Total' in sd:
                sex = 'both'
            elif 'below poverty level' in sd:
                pov = 'below'
            elif 'above poverty level' in sd:
                pov = 'above'
            
        col_map[e.unique_id] = Dimensions( sex, parse_age(sd), pov)

    return col_map 

def get_raceeth(pkg, re_code, re_name):
    """Get a single race/ethnicity iteration, and process it so 
    it can be merged with others. """
    
    pov_sex_age_url = pkg.reference('pov_sex_age_template').resolved_url
    url = pov_sex_age_url.interpolate(context={'iteration':re_code.strip()})
    
    psa = url.dataframe()
    psa = psa[psa.index.isin(sd_puma_tracts)]
    
    
    
    assert len(psa) > 0
    
    col_map= make_col_map(psa)
    
    t = psa.drop(columns=['STUSAB', 'COUNTY','NAME'])
    t = t.stack().to_frame().reset_index()
    t.columns = ['geoid','col_name','value']
    
    t['m_type'] = t.col_name.apply(lambda v: 'margin' if '_m90' in v else 'est')
    t['col_name'] = t.col_name.apply(lambda v: v.replace('_m90',''))
    t = t.set_index(['geoid','col_name','m_type']).unstack()
    t['sex'] = [col_map[str(i[1]).replace('_m90','')].sex for i,r in t.iterrows() ]
    t['raceeth'] = re_name
    t['age_range'] = [col_map[str(i[1]).replace('_m90','')].age_range for i,r in t.iterrows() ]
    t['pov'] = [col_map[str(i[1]).replace('_m90','')].pov for i,r in t.iterrows() ]

    t.columns = [ '_'.join(e) if e[1] else e[0] for e in zip(*zip(*t.columns)) ]
    
    return t[['value_est',  'value_margin', 'sex', 'raceeth', 'age_range', 'pov']]

def concat_race_eth(pkg):
    

    frames= []

    tn = tqdm_notebook(list(race_iterations.itertuples()))
    
    for e in tn:
        tn.set_description(e.raceeth)
        frames.append(get_raceeth(pkg, e.code, e.raceeth))
 
    return pd.concat(frames)

df = concat_race_eth(pkg).reset_index()

df['age_min'] = df.age_range.apply(lambda v: v.left)
df['age_max'] = df.age_range.apply(lambda v: v.right)
df['age_range'] = df.age_range.apply(lambda v: "{:02d}-{:03d}".format(v.left, v.right))
df.head()
# Select just the non-overlaping entries
#df = df[(df.sex != 'both') & (df.pov != 'all')& (df.raceeth != 'white') & (~df.age_range.isnull())].copy()


assert df.value_est.sum() > 3_100_000 # Ought to be close to CA population 

In [9]:
df[(df.age_range.isnull())].head()
df['overlapping'] = 0 
df['overlapping'] = df.overlapping.where(df.pov != 'all', 1)
df['overlapping'] = df.overlapping.where(df.sex != 'both', 1)
df['overlapping'] = df.overlapping.where(df.age_range !=  '00-120', 1)
df['overlapping'] = df.overlapping.where(df.raceeth != 'white', 1)
df['overlapping'] = df.overlapping.where(df.raceeth != 'all', 1)
df[df.overlapping == 0].value_est.sum()

3452837

In [10]:
df[(df.pov == 'all') & (df.raceeth=='all')].head()

,geoid,col_name,value_est,value_margin,sex,raceeth,age_range,pov,age_min,age_max,overlapping
333468,14000US06073000100,B17001_001,2773,185,both,all,00-120,all,0,120,1
333527,14000US06073000201,B17001_001,2158,241,both,all,00-120,all,0,120,1
333586,14000US06073000202,B17001_001,4769,414,both,all,00-120,all,0,120,1
333645,14000US06073000300,B17001_001,4790,374,both,all,00-120,all,0,120,1
333704,14000US06073000400,B17001_001,3629,287,both,all,00-120,all,0,120,1


In [11]:
df[df.col_name == 'B17001_001'].value_est.sum()

3172544

In [12]:
%mt_materialize df '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-rasp-1' 

{
    "df_name": "df",
    "path": "/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-rasp-1/df.csv"
}


In [13]:

%mt_materialize_all '/Users/eric/Library/Application Support/metapack/_materialized_data/sandiegodata.org-rasp-1'


[]


In [14]:

%mt_show_metatab



Declare: metatab-latest
Section: Resources


In [15]:

%mt_show_libdirs



[]
